In [2]:
import pandas as pd
import glob
import numpy as np
pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#import matplotlib.pyplot as plt
import requests
from geopy.geocoders import Nominatim
import time

In [3]:
glob.glob('./*/*/*.*')

['.\\Tables_brutes\\Aeroport_francais\\d9c5dbbe269cee68a13ccbe7e7eda7f3728b39d5f62cd6e9a7ad5272fd5678.xls',
 '.\\Tables_brutes\\Aeroport_francais\\liste_aeroports.csv',
 '.\\Tables_brutes\\Base Nationale des Lieux de Covoiturage en France\\base-nationale-des-lieux-de-covoiturage-france.csv',
 '.\\Tables_brutes\\Base nationale des lieux de stationnement en France\\mobilityref-france-base-nationale-des-lieux-de-stationnement.csv',
 '.\\Tables_brutes\\Communes_adjacentes\\communes-departement-region.csv',
 '.\\Tables_brutes\\Communes_adjacentes\\communes_adjacentes_2022.csv',
 '.\\Tables_brutes\\Communes_adjacentes\\test_communes.csv',
 '.\\Tables_brutes\\Contour_communes_2023\\communes-1000m.geojson',
 '.\\Tables_brutes\\Contour_communes_2023\\communes-100m.geojson',
 '.\\Tables_brutes\\Contour_communes_2023\\mairies.geojson',
 '.\\Tables_brutes\\Correspondance Code INSEE - Code Postal\\correspondance-code-insee-code-postal.csv',
 '.\\Tables_brutes\\Etablissement_publics_cooperation_inte

In [3]:
# Utilisation de l'API geo.API.gouv.fr
# Retourne les infos sur la commune en fonction de son nom
def info_communes(nom):
    endpoint = "https://geo.api.gouv.fr/communes?nom="
    # Create the web API’s URL
    authenticated_endpoint = endpoint+nom
    # Get the web API’s reply to the endpoint
    try: 
        api_response = requests.get(authenticated_endpoint).json()
    except:
        api_response = [{'nom': None, 'code': None, 'codeDepartement': None, 'siren': None, 'codeEpci': None, 'codeRegion': None, 'codesPostaux': [None], 'population': 0, '_score': 0}]
    if not api_response : 
        api_response = [{'nom': None, 'code': None, 'codeDepartement': None, 'siren': None, 'codeEpci': None, 'codeRegion': None, 'codesPostaux': [None], 'population': 0, '_score': 0}]
    print(api_response[0]['code'])
    return api_response

# Retourne les infos sur l'adresse et la commune en fonction des coordonnées lat , lon
def get_rev_commune(lat,lon):
    endpoint = "https://api-adresse.data.gouv.fr/reverse/?"
    # Create the web API’s URL
    authenticated_endpoint = endpoint+'lon='+str(lon)+'&lat='+str(lat)
    # Get the web API’s reply to the endpoint
    api_response = requests.get(authenticated_endpoint).json()
    # print(api_response)
    return api_response

In [4]:
# Utilisation de l'API open Street Map Nominatim
def get_location_info(address):
    geolocator = Nominatim(user_agent="my_geocoder")
    location = geolocator.geocode(address, addressdetails=True)

    if location:
        latitude = location.latitude
        longitude = location.longitude
        raw_address = location.raw.get('address', {})
        country = raw_address.get('country', None)
        return latitude, longitude, raw_address, country
    else:
        return None, None,None,None

def get_rev_location_info(lat,lon):
     geolocator = Nominatim(user_agent="my_geocoder")
     location = geolocator.reverse(f"{lat}, {lon}")
     if location:
        latitude = location.latitude
        longitude = location.longitude
        raw_address = location.raw.get('address', {})
        country = raw_address.get('country', None)
        return latitude, longitude, raw_address, country
     else:
        return 0.0,0.0,'not found','not found'

# Analyse des datas

## Bornes de recharge électrique en France

https://public.opendatasoft.com/explore/dataset/mobilityref-france-irve-220/information/?disjunctive.nom_amenageur&disjunctive.nom_operateur&disjunctive.nom_enseigne&disjunctive.implantation_station&disjunctive.prise_type_ef&disjunctive.prise_type_2&disjunctive.prise_type_combo_ccs&disjunctive.prise_type_chademo&disjunctive.prise_type_autre&disjunctive.gratuit&disjunctive.paiement_acte&disjunctive.paiement_cb&disjunctive.paiement_autre&disjunctive.condition_acces&disjunctive.reservation&disjunctive.raccordement&disjunctive.com_arm_name&disjunctive.nom_epci&disjunctive.dep_name&disjunctive.reg_name&basemap=jawg.light&location=2,0.08175,8.27828&dataChart=eyJxdWVyaWVzIjpbeyJjaGFydHMiOlt7InR5cGUiOiJsaW5lIiwiZnVuYyI6IkNPVU5UIiwieUF4aXMiOiJuYnJlX3BkYyIsInNjaWVudGlmaWNEaXNwbGF5Ijp0cnVlLCJjb2xvciI6IiNGRjUxNUEifV0sInhBeGlzIjoiZGF0ZV9taXNlX2VuX3NlcnZpY2UiLCJtYXhwb2ludHMiOm51bGwsInRpbWVzY2FsZSI6Im1vbnRoIiwic29ydCI6IiIsImNvbmZpZyI6eyJkYXRhc2V0IjoibW9iaWxpdHlyZWYtZnJhbmNlLWlydmUtMjIwIiwib3B0aW9ucyI6eyJkaXNqdW5jdGl2ZS5ub21fYW1lbmFnZXVyIjp0cnVlLCJkaXNqdW5jdGl2ZS5ub21fb3BlcmF0ZXVyIjp0cnVlLCJkaXNqdW5jdGl2ZS5ub21fZW5zZWlnbmUiOnRydWUsImRpc2p1bmN0aXZlLmltcGxhbnRhdGlvbl9zdGF0aW9uIjp0cnVlLCJkaXNqdW5jdGl2ZS5wcmlzZV90eXBlX2VmIjp0cnVlLCJkaXNqdW5jdGl2ZS5wcmlzZV90eXBlXzIiOnRydWUsImRpc2p1bmN0aXZlLnByaXNlX3R5cGVfY29tYm9fY2NzIjp0cnVlLCJkaXNqdW5jdGl2ZS5wcmlzZV90eXBlX2NoYWRlbW8iOnRydWUsImRpc2p1bmN0aXZlLnByaXNlX3R5cGVfYXV0cmUiOnRydWUsImRpc2p1bmN0aXZlLmdyYXR1aXQiOnRydWUsImRpc2p1bmN0aXZlLnBhaWVtZW50X2FjdGUiOnRydWUsImRpc2p1bmN0aXZlLnBhaWVtZW50X2NiIjp0cnVlLCJkaXNqdW5jdGl2ZS5wYWllbWVudF9hdXRyZSI6dHJ1ZSwiZGlzanVuY3RpdmUuY29uZGl0aW9uX2FjY2VzIjp0cnVlLCJkaXNqdW5jdGl2ZS5yZXNlcnZhdGlvbiI6dHJ1ZSwiZGlzanVuY3RpdmUucmFjY29yZGVtZW50Ijp0cnVlLCJkaXNqdW5jdGl2ZS5jb21fYXJtX25hbWUiOnRydWUsImRpc2p1bmN0aXZlLm5vbV9lcGNpIjp0cnVlLCJkaXNqdW5jdGl2ZS5kZXBfbmFtZSI6dHJ1ZSwiZGlzanVuY3RpdmUucmVnX25hbWUiOnRydWV9fX1dLCJkaXNwbGF5TGVnZW5kIjp0cnVlLCJhbGlnbk1vbnRoIjp0cnVlLCJ0aW1lc2NhbGUiOiIifQ%3D%3D

In [5]:
df_elec = pd.read_csv('.\\Tables_brutes\\Fichier consolidé des Bornes de Recharge pour Véhicules Électriques (IRVE) – schéma 2.2.0\\mobilityref-france-irve-220.csv',sep=';')

In [6]:
df_elec.shape

(64304, 63)

In [7]:
df_elec[8:12]

,Nom aménageur,SIREN aménageur,Contact aménageur,Nom opérateur,Contact opérateur,Téléphone opérateur,Nom enseigne,ID station itinérance,ID station local,Nom station,...,latitude,point_geo,Nom de la commune ou de l'arrondissement municipal,Code de la commune ou de l'arrondissement,Code EPCI,Nom de l'EPCI,Code Officiel Département,Nom Officiel Département,Code Officiel Région,Nom Officiel Région
8,SPBR1,NaN,NaN,SPBR1,contact@reseau-eborn.fr,33423100350,eborn,FREBNPKUMQMM,KUMQMM,"Rumilly, Corniche de la Néphaz",...,45.866400,"45.8664, 5.94101",Rumilly,74225,247400740.0,CC Rumilly Terre de Savoie,74,Haute-Savoie,84.0,Auvergne-Rhône-Alpes
9,SYANE - Haute-Savoie accélérée,NaN,NaN,Freshmile SAS,web@freshmile.com,0369246732,Freshmile,FREBNPLNG7XIKQU2,LNG7XIKQU2,"Taninges (Praz De Lys), Route de la Ramaz",...,46.144208,"46.144208, 6.591829",Taninges,74276,200034098.0,CC des Montagnes du Giffre,74,Haute-Savoie,84.0,Auvergne-Rhône-Alpes
10,SPBR1,NaN,NaN,SPBR1,contact@reseau-eborn.fr,33423100350,eborn,FREBNPDGMDE60VKP,DGMDE60VKP,"Unieux, Parking rue Marechal Leclerc",...,45.396748,"45.396748, 4.263238",Unieux,42316,244200770.0,Saint-Etienne Métropole,42,Loire,84.0,Auvergne-Rhône-Alpes
11,SPBR1,NaN,NaN,SPBR1,contact@reseau-eborn.fr,33423100350,eborn,FREBNPSBBFTCPVCD,SBBFTCPVCD,"Lyaud, Eglise",...,46.337150,"46.33715, 6.52257",Lyaud,74157,200067551.0,CA Thonon Agglomération,74,Haute-Savoie,84.0,Auvergne-Rhône-Alpes


In [8]:
df_elec.columns

Index(['Nom aménageur', 'SIREN aménageur', 'Contact aménageur',
       'Nom opérateur', 'Contact opérateur', 'Téléphone opérateur',
       'Nom enseigne', 'ID station itinérance', 'ID station local',
       'Nom station', 'Implantation station', 'Adresse station',
       'Code INSEE commune', 'Coordonnées', 'Nombre Points de charge',
       'ID PDC itinérance', 'ID PDC local', 'Puissance nominale',
       'Prise type E/F', 'Prise type 2', 'Prise type combo ccs',
       'Prise type chademo', 'Prise type autre', 'Gratuit', 'Paiement acte',
       'Paiement CB', 'Paiement autre', 'Tarification', 'Condition accès',
       'Réservation', 'Horaires', 'Accessibilité PMR', 'Restriction gabarit',
       'Station deux roues', 'Raccordement', 'Num PDL', 'Date mise en service',
       'Observations', 'Date de Mise à Jour', 'cable_t2_attache',
       'Dernière modification', 'datagouv dataset id', 'datagouv resource id',
       'datagouv organization or owner', 'created_at',
       'consolidated_lo

In [9]:
df_elec.dtypes

Nom aménageur                 object
SIREN aménageur              float64
Contact aménageur             object
Nom opérateur                 object
Contact opérateur             object
                              ...   
Nom de l'EPCI                 object
Code Officiel Département     object
Nom Officiel Département      object
Code Officiel Région         float64
Nom Officiel Région           object
Length: 63, dtype: object

In [10]:
df_elec['ID PDC itinérance'].unique()

array(['FREBNEBPUT1', 'FREBNECJVC2', 'FREBNECMZH2', ..., 'FRUBIE10006653',
       'FRUBIE10017771', 'FRUBIE10036695'], dtype=object)

In [11]:
df_elec['ID PDC local'].unique()

array(['BPUT1', 'CJVC2', 'CMZH2', ..., 'FRTNME089127900',
       'FRTNME089903950', 'FRTNME099901800'], dtype=object)

In [12]:
selected_col = [ 'Nom opérateur', 'Nom enseigne', 
        
        'ID station itinérance', 'ID station local','Nom station', 'Implantation station', 
        'Date mise en service','Nombre Points de charge',
               
                
       'Gratuit','Paiement CB','Paiement autre','Tarification', 'Condition accès',
       'Réservation', 'Horaires', 'Accessibilité PMR', 'Restriction gabarit',
       'Station deux roues',

         'ID PDC itinérance', 'ID PDC local',
        'Puissance nominale', 'Prise type E/F', 'Prise type 2', 'Prise type combo ccs', 'Prise type chademo',
         'Prise type autre', 

        'Adresse station',  
        'consolidated_longitude', 'consolidated_latitude',      
        "Code de la commune ou de l'arrondissement","Nom de la commune ou de l'arrondissement municipal",
        'Code EPCI',"Nom de l'EPCI",  
        'Code Officiel Département', 'Nom Officiel Département', 
        'Code Officiel Région','Nom Officiel Région',
        'Code INSEE commune',
        
        'Coordonnées', 
        'Paiement acte', 'Raccordement', 'Num PDL', 
        'Observations', 'Date de Mise à Jour', 'cable_t2_attache','Dernière modification', 
        'Nom aménageur',
        'consolidated_code_postal', 'consolidated_commune','consolidated_is_lon_lat_correct',
        'coord', 'longitude', 'latitude','point_geo', 
                  
               ]

In [13]:
df_elec[df_elec["Code de la commune ou de l'arrondissement"]=='13201']

,Nom aménageur,SIREN aménageur,Contact aménageur,Nom opérateur,Contact opérateur,Téléphone opérateur,Nom enseigne,ID station itinérance,ID station local,Nom station,...,latitude,point_geo,Nom de la commune ou de l'arrondissement municipal,Code de la commune ou de l'arrondissement,Code EPCI,Nom de l'EPCI,Code Officiel Département,Nom Officiel Département,Code Officiel Région,Nom Officiel Région
1983,NaN,NaN,kontakt@lidl.de,NaN,kontakt@lidl.de,+(33)-(800)-900343,LFR0893EVCP02,FRLDLE0000334,FRLDLE0000334,LFR0893EVCP02,...,43.291853,"43.291853, 5.372331",Marseille 1er Arrondissement,13201,200054807.0,Métropole d'Aix-Marseille-Provence,13,Bouches-du-Rhône,93.0,Provence-Alpes-Côte d'Azur
2175,INDIGO FRANCE,NaN,support@alizecharge.fr,Bouygues E&S,support@alizecharge.fr,0805021480,INDIGO FRANCE,FRP07P13202004,NaN,MARSEILLE - République,...,43.298984,"43.2989844, 5.3720389",Marseille 1er Arrondissement,13201,200054807.0,Métropole d'Aix-Marseille-Provence,13,Bouches-du-Rhône,93.0,Provence-Alpes-Côte d'Azur
4348,INDIGO FRANCE,NaN,support@alizecharge.fr,Bouygues E&S,support@alizecharge.fr,0805021480,INDIGO FRANCE,FRP07P13201001,NaN,MARSEILLE - Estienne d’Orves,...,43.292248,"43.292248, 5.372177",Marseille 1er Arrondissement,13201,200054807.0,Métropole d'Aix-Marseille-Provence,13,Bouches-du-Rhône,93.0,Provence-Alpes-Côte d'Azur
4349,INDIGO FRANCE,NaN,support@alizecharge.fr,Bouygues E&S,support@alizecharge.fr,0805021480,INDIGO FRANCE,FRP07P13201001,NaN,MARSEILLE - Estienne d’Orves,...,43.292248,"43.292248, 5.372177",Marseille 1er Arrondissement,13201,200054807.0,Métropole d'Aix-Marseille-Provence,13,Bouches-du-Rhône,93.0,Provence-Alpes-Côte d'Azur
6272,INDIGO FRANCE,NaN,support@alizecharge.fr,Bouygues E&S,support@alizecharge.fr,0805021480,INDIGO FRANCE,FRP07P13202004,NaN,MARSEILLE - République,...,43.298984,"43.2989844, 5.3720389",Marseille 1er Arrondissement,13201,200054807.0,Métropole d'Aix-Marseille-Provence,13,Bouches-du-Rhône,93.0,Provence-Alpes-Côte d'Azur
8857,INDIGO FRANCE,NaN,support@alizecharge.fr,Bouygues E&S,support@alizecharge.fr,0805021480,INDIGO FRANCE,FRP07P13202004,NaN,MARSEILLE - République,...,43.298984,"43.2989844, 5.3720389",Marseille 1er Arrondissement,13201,200054807.0,Métropole d'Aix-Marseille-Provence,13,Bouches-du-Rhône,93.0,Provence-Alpes-Côte d'Azur
8858,INDIGO FRANCE,NaN,support@alizecharge.fr,Bouygues E&S,support@alizecharge.fr,0805021480,INDIGO FRANCE,FRP07P13202004,NaN,MARSEILLE - République,...,43.298984,"43.2989844, 5.3720389",Marseille 1er Arrondissement,13201,200054807.0,Métropole d'Aix-Marseille-Provence,13,Bouches-du-Rhône,93.0,Provence-Alpes-Côte d'Azur
10593,MAMP - Bornes AC recharge accélérée 22kW,NaN,NaN,Freshmile SAS,web@freshmile.com,0369246732,Freshmile,FRM13PCCCQNBES7L,CCCQNBES7L,"Marseille-1E, Place des Marseillaises",...,43.301561,"43.301561, 5.381055",Marseille 1er Arrondissement,13201,200054807.0,Métropole d'Aix-Marseille-Provence,13,Bouches-du-Rhône,93.0,Provence-Alpes-Côte d'Azur
10594,MAMP - Bornes AC recharge accélérée 22kW,NaN,NaN,Freshmile SAS,web@freshmile.com,0369246732,Freshmile,FRM13PALJ6PPQPWI,ALJ6PPQPWI,"Marseille-1E, 11 Boulevard Montricher",...,43.304651,"43.304651, 5.39319",Marseille 1er Arrondissement,13201,200054807.0,Métropole d'Aix-Marseille-Provence,13,Bouches-du-Rhône,93.0,Provence-Alpes-Côte d'Azur
10595,Aix-Marseille Provence Métropole,200054807.0,larecharge@ampmetropole.fr,EVzen (SMEG Développement),support@evzen.com,37792050403,larecharge,FRM13PALJ6PPQPWI,NaN,"Marseille-1E, 11 Boulevard Montricher",...,43.304651,"43.304651, 5.39319",Marseille 1er Arrondissement,13201,200054807.0,Métropole d'Aix-Marseille-Provence,13,Bouches-du-Rhône,93.0,Provence-Alpes-Côte d'Azur


### Analyse du fichier et des valeurs manquantes

In [33]:
# Valeurs uniques par colonne
df_elec_analyse = []
manquantes = df_elec.isna().sum()
for t in df_elec.columns:
    row = {}
    row['Variable']=t
    row['type']= df_elec[t].dtypes
    row['Nb_val_unique']=len(df_elec[t].unique())
    row['Nb_manquant']=manquantes[t]
    df_elec_analyse.append(row)
df_elec_analyse = pd.DataFrame(df_elec_analyse)
df_elec_analyse.sort_values(by='Nb_manquant',ascending=False)

,Variable,type,Nb_val_unique,Nb_manquant
1,SIREN aménageur,float64,608,43077
47,consolidated_code_postal,float64,2649,41103
35,Num PDL,object,2797,38614
48,consolidated_commune,object,4200,37326
34,Raccordement,object,3,35250
27,Tarification,object,214,33502
12,Code INSEE commune,object,4413,32790
37,Observations,object,126,32150
39,cable_t2_attache,object,8,28767
8,ID station local,object,12850,21829


In [15]:
# Valeurs manquantes
plt.figure(figsize=(22,9))
plt.bar(x= df_elec_analyse.sort_values(by='Nb_manquant',ascending=False)['Variable'], height = df_elec_analyse.sort_values(by='Nb_manquant',ascending=False)['Nb_manquant'])
plt.xticks(rotation=90)
plt.grid()
plt.ylabel('Valeurs Manquantes (-)')

NameError: name 'plt' is not defined

In [16]:
# Station ou le code de la commune est manquante
pd.options.display.max_colwidth = 1000
print(len(df_elec[df_elec["Code de la commune ou de l'arrondissement"].isna()]))
df_elec.loc[df_elec["Code de la commune ou de l'arrondissement"].isna(),['Adresse station','consolidated_longitude', 'consolidated_latitude',
        'Code INSEE commune',"Nom de la commune ou de l'arrondissement municipal",'consolidated_code_postal', 'consolidated_commune'  
               ]]

100


,Adresse station,consolidated_longitude,consolidated_latitude,Code INSEE commune,Nom de la commune ou de l'arrondissement municipal,consolidated_code_postal,consolidated_commune
884,PALAGACCIO 20200 VILLE-DI-PIETRABUGNO,9.460000,42.720000,2B353,NaN,20200.0,Ville-di-Pietrabugno
1515,8 RUE EDOUARD NIGNON 44000 NANTES,0.000000,47.276100,44109,NaN,44000.0,Nantes
1533,2 RUE LOUIS BLERIOT Kyriad Limoges Feytiat 87220 FEYTIAT,0.000000,0.000000,87065,NaN,87220.0,Feytiat
1631,2 RUE LOUIS BLERIOT Kyriad Limoges Feytiat 87220 FEYTIAT,0.000000,0.000000,87065,NaN,87220.0,Feytiat
1695,14 Avenue Du Pré-Aily 4031 Liège,5.569035,50.598908,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
59120,3 Rue De Mons 7080 Frameries,3.902843,50.420024,NaN,NaN,NaN,NaN
59844,Rue D'anvers 7041 Quévy,4.032694,50.377567,NaN,NaN,NaN,NaN
62278,2 RUE LOUIS BLERIOT Kyriad Limoges Feytiat 87220 FEYTIAT,0.000000,0.000000,87065,NaN,87220.0,Feytiat
62666,Avenue Des Dessus De Lives 5101 Namur,4.923860,50.461821,NaN,NaN,NaN,NaN


In [17]:
df_elec_analyse.sort_values(by='Nb_val_unique',ascending=False)

,Variable,type,Nb_val_unique,Nb_manquant
15,ID PDC itinérance,object,58383,0
16,ID PDC local,object,39700,20394
7,ID station itinérance,object,19268,0
13,Coordonnées,object,18874,0
9,Nom station,object,18837,0
...,...,...,...,...
20,Prise type combo ccs,int64,2,0
19,Prise type 2,int64,2,0
18,Prise type E/F,int64,2,0
33,Station deux roues,int64,2,0


In [ ]:
# Valeurs uniques
plt.figure(figsize=(22,9))
plt.bar(x= df_elec_analyse.sort_values(by='Nb_val_unique',ascending=False)['Variable'], height =df_elec_analyse.sort_values(by='Nb_val_unique',ascending=False)['Nb_val_unique'])
plt.xticks(rotation=90)
plt.grid()
plt.ylabel('Nb Valeurs Uniques (-)')

In [18]:
for c in df_elec.columns:
    print(c)
    print(df_elec[c].unique().shape)
    # print(df_elec[c].unique())
    print('-----------------------------')

Nom aménageur
(2743,)
-----------------------------
SIREN aménageur
(622,)
-----------------------------
Contact aménageur
(466,)
-----------------------------
Nom opérateur
(216,)
-----------------------------
Contact opérateur
(245,)
-----------------------------
Téléphone opérateur
(278,)
-----------------------------
Nom enseigne
(3971,)
-----------------------------
ID station itinérance
(19268,)
-----------------------------
ID station local
(12991,)
-----------------------------
Nom station
(18837,)
-----------------------------
Implantation station
(5,)
-----------------------------
Adresse station
(18414,)
-----------------------------
Code INSEE commune
(4447,)
-----------------------------
Coordonnées
(18874,)
-----------------------------
Nombre Points de charge
(55,)
-----------------------------
ID PDC itinérance
(58383,)
-----------------------------
ID PDC local
(39700,)
-----------------------------
Puissance nominale
(80,)
-----------------------------
Prise type E/F


In [ ]:
for c in df_elec.columns:
    print(c)
    print(df_elec[c].unique().shape)
    print(df_elec[c].unique())
    print('-----------------------------')

### Traitement des valeurs manquantes

On ne va s'intéresser qu'à la France Métropolitaine, on commence donc par supprimer les autres régions

In [19]:
df_elec['Nom Officiel Région'].unique()

array(['Auvergne-Rhône-Alpes', "Provence-Alpes-Côte d'Azur",
       'Centre-Val de Loire', 'Pays de la Loire', 'Occitanie',
       'Normandie', 'Grand Est', 'Bourgogne-Franche-Comté', 'Bretagne',
       'Île-de-France', 'Hauts-de-France', 'Nouvelle-Aquitaine', 'Corse',
       nan, 'La Réunion', 'Martinique', 'Guadeloupe', 'Guyane'],
      dtype=object)

In [20]:
df_elec['Nom Officiel Région'].value_counts()

Nom Officiel Région
Île-de-France                 11439
Auvergne-Rhône-Alpes           8395
Occitanie                      6813
Provence-Alpes-Côte d'Azur     6013
Hauts-de-France                5556
Grand Est                      5453
Bretagne                       4255
Normandie                      3849
Nouvelle-Aquitaine             3703
Pays de la Loire               3002
Bourgogne-Franche-Comté        2631
Centre-Val de Loire            2506
Corse                           342
La Réunion                      225
Guadeloupe                       15
Martinique                        5
Guyane                            2
Name: count, dtype: int64

In [21]:
# Nous allons traiter que la France métropolitaine
df_elec= df_elec[(df_elec['Nom Officiel Région']!='Guyane') & (df_elec['Nom Officiel Région']!='Martinique') & (df_elec['Nom Officiel Région']!='Guadeloupe') & (df_elec['Nom Officiel Région']!='La Réunion') & (df_elec['Nom Officiel Région']!='Corse')]

In [22]:
df_elec.shape

(63715, 63)

Détermination des infos manquantes concernant la localisation

In [ ]:
loc_col =['Adresse station',  
        'consolidated_longitude', 'consolidated_latitude',      
        "Code de la commune ou de l'arrondissement","Nom de la commune ou de l'arrondissement municipal",
        'Code EPCI',"Nom de l'EPCI",  
        'Code Officiel Département', 'Nom Officiel Département', 
        'Code Officiel Région','Nom Officiel Région',
          'Code INSEE commune',
         ]

Code INSEE Commune

In [23]:
# Il n'y a pas de valeurs manquantes dans les adresses
print(len(df_elec[df_elec['Adresse station'].isna()]))

0


In [24]:
# Il manque beacoup de valeurs dans ce champ
print(len(df_elec[df_elec["Code INSEE commune"].isna()]))

32790


In [25]:
# IL ne manque pas beacoup de valeurs dans ce chanmps
print(len(df_elec[df_elec["Code de la commune ou de l'arrondissement"].isna()]))

100


In [26]:
df_elec[df_elec["Code de la commune ou de l'arrondissement"]=='75114']

,Nom aménageur,SIREN aménageur,Contact aménageur,Nom opérateur,Contact opérateur,Téléphone opérateur,Nom enseigne,ID station itinérance,ID station local,Nom station,...,latitude,point_geo,Nom de la commune ou de l'arrondissement municipal,Code de la commune ou de l'arrondissement,Code EPCI,Nom de l'EPCI,Code Officiel Département,Nom Officiel Département,Code Officiel Région,Nom Officiel Région
647,TotalEnergies Charging Services,844192443.0,supervision-ev.france@totalenergies.com,TotalEnergies Charging Services,supervision-ev.france@totalenergies.com,0185169402,Belib',FRV75P901401,FR*V75*P9014*01,Paris | Avenue du Maine 173,...,48.830860,"48.83086, 2.325895",Paris 14e Arrondissement,75114,200054781.0,Métropole du Grand Paris,75,Paris,11.0,Île-de-France
686,TotalEnergies Charging Services,844192443.0,supervision-ev.france@totalenergies.com,TotalEnergies Charging Services,supervision-ev.france@totalenergies.com,0185169402,Belib',FRV75PPX1410,FR*V75*PPX14*10,Paris | Rue Henri Barbusse 66,...,48.839120,"48.83912, 2.337229",Paris 14e Arrondissement,75114,200054781.0,Métropole du Grand Paris,75,Paris,11.0,Île-de-France
4227,TotalEnergies Charging Services,844192443.0,supervision-ev.france@totalenergies.com,TotalEnergies Charging Services,supervision-ev.france@totalenergies.com,0185169402,Belib',FRV75P901403,FR*V75*P9014*03,Paris | Boulevard Edgar Quinet 9,...,48.841198,"48.841198, 2.323891",Paris 14e Arrondissement,75114,200054781.0,Métropole du Grand Paris,75,Paris,11.0,Île-de-France
4268,TotalEnergies Charging Services,844192443.0,supervision-ev.france@totalenergies.com,TotalEnergies Charging Services,supervision-ev.france@totalenergies.com,0185169402,Belib',FRV75PPX1402,FR*V75*PPX14*02,Paris | Boulevard de Port-Royal 97,...,48.838312,"48.8383124, 2.3408664",Paris 14e Arrondissement,75114,200054781.0,Métropole du Grand Paris,75,Paris,11.0,Île-de-France
4269,TotalEnergies Charging Services,844192443.0,supervision-ev.france@totalenergies.com,TotalEnergies Charging Services,supervision-ev.france@totalenergies.com,0185169402,Belib',FRV75PPX1412,FR*V75*PPX14*12,Paris | Rue Raymond Losserand 145,...,48.831260,"48.83126, 2.313088",Paris 14e Arrondissement,75114,200054781.0,Métropole du Grand Paris,75,Paris,11.0,Île-de-France
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57646,TotalEnergies Charging Services,844192443.0,supervision-ev.france@totalenergies.com,TotalEnergies Charging Services,supervision-ev.france@totalenergies.com,0185169402,Belib',FRV75PPX1413,FR*V75*PPX14*13,Paris | Boulevard Jourdan 26,...,48.820760,"48.8207604, 2.3370783",Paris 14e Arrondissement,75114,200054781.0,Métropole du Grand Paris,75,Paris,11.0,Île-de-France
59338,TotalEnergies Charging Services,844192443.0,supervision-ev.france@totalenergies.com,TotalEnergies Charging Services,supervision-ev.france@totalenergies.com,0141354000,TotalEnergies Charge Rapide,FRHPCPNF080179,FR*HPC*PNF080179,RELAIS DE LA PORTE DE CHATILLON,...,48.823581,"48.823581, 2.316345",Paris 14e Arrondissement,75114,200054781.0,Métropole du Grand Paris,75,Paris,11.0,Île-de-France
60614,TotalEnergies Charging Services,844192443.0,supervision-ev.france@totalenergies.com,TotalEnergies Charging Services,supervision-ev.france@totalenergies.com,0185169402,Belib',FRV75PPX1412,FR*V75*PPX14*12,Paris | Rue Raymond Losserand 145,...,48.831260,"48.83126, 2.313088",Paris 14e Arrondissement,75114,200054781.0,Métropole du Grand Paris,75,Paris,11.0,Île-de-France
64056,TotalEnergies Charging Services,844192443.0,supervision-ev.france@totalenergies.com,TotalEnergies Charging Services,supervision-ev.france@totalenergies.com,0185169402,Belib',FRV75PPX1411,FR*V75*PPX14*11,Paris | Avenue Jean Moulin 34,...,48.826681,"48.8266807, 2.3237355",Paris 14e Arrondissement,75114,200054781.0,Métropole du Grand Paris,75,Paris,11.0,Île-de-France


In [27]:
df_elec["Code de la commune ou de l'arrondissement"].value_counts()

Code de la commune ou de l'arrondissement
75108    1097
31555     846
75116     502
75110     418
75115     298
         ... 
06128       1
24552       1
25542       1
17242       1
40233       1
Name: count, Length: 7457, dtype: int64

In [28]:
# Lignes ou la cellule Code de commune est NaN
print(len(df_elec.loc[df_elec["Code de la commune ou de l'arrondissement"].isna(),loc_col]))
df_elec.loc[df_elec["Code de la commune ou de l'arrondissement"].isna(),loc_col]

NameError: name 'loc_col' is not defined

In [29]:
def check_code_commune(dfx):
    if pd.notna(dfx["Code de la commune ou de l'arrondissement"]):
        code_INSEE = dfx["Code de la commune ou de l'arrondissement"]
    elif pd.notna(dfx["Code INSEE commune"]):
        code_INSEE = dfx["Code INSEE commune"]
    elif pd.notna(dfx["consolidated_code_postal"]):
        code_INSEE = info_communes(dfx["consolidated_code_postal"])
    else:
        try:
            infos_adresses = get_location_info(dfx['Adresse station'])
            # print(infos_adresses)
            code_postal = infos_adresses[2]['city']
            code_INSEE=info_communes[0]['code']
        except:
            code_INSEE=None
    return code_INSEE

In [30]:
# On chercher le code commune d'abord dans le tableau, puis avec l'API.gouv.fr
def check_code_commune(dfx):
    if pd.notna(dfx["Code de la commune ou de l'arrondissement"]):
        code_INSEE = dfx["Code de la commune ou de l'arrondissement"]
    elif pd.notna(dfx["Code INSEE commune"]):
        code_INSEE = dfx["Code INSEE commune"]
    elif pd.notna(dfx["consolidated_code_postal"]):
        code_INSEE = info_communes(dfx["consolidated_code_postal"])
    else:
        try:
            infos_adresses = get_location_info(dfx['Adresse station'])
            # print(infos_adresses)
            code_postal = infos_adresses[2]['city']
            code_INSEE=info_communes[0]['code']
        except:
            code_INSEE=None
    return code_INSEE
df_elec['code_commune_n']=df_elec.apply(lambda x: check_code_commune(x), axis=1)

In [ ]:
# Ensuite on recherche par lat lon 
for i, x in df_elec[(df_elec['code_commune_n'].isna()) & (df_elec['latitude'].notna()) & (df_elec['longitude'].notna())].iterrows():
    print(i)
    try: 
        df_elec.loc[i,'code_commune_n'] = get_rev_commune(df_elec.loc[i,'latitude'],df_elec.loc[i,'longitude'])['features'][0]['properties']['citycode']
    except:
        pass

In [31]:
i= 10656
get_rev_commune(df_elec.loc[i,'latitude'],df_elec.loc[i,'longitude'])

{'type': 'FeatureCollection',
 'version': 'draft',
 'features': [],
 'attribution': 'BAN',
 'licence': 'ETALAB-2.0',
 'center': [4.3438463, 50.84746188],
 'limit': 1}

In [ ]:
#df_elec.to_csv(r'C:\Users\nimod\OneDrive - Université de Technologie de Troyes\Fil_rouge\Tables_traitees\Stations_electriques_avec_code_commune.csv',sep=';',index=False)

In [ ]:
df_elec[df_elec['code_commune_n'].isna()]

In [ ]:
# Lignes ou la cellule Code commune checked == 0
loc_col.append('code_commune_n')
print(len(df_elec.loc[df_elec["code_commune_n"]==0,loc_col]))
df_elec.loc[df_elec["code_commune_n"]==0,loc_col]

In [ ]:
# On va supprimer les lignes où le code_commune_checked==0 car ce n'est pas en France métrolpolitaine
df_elec =df_elec[df_elec["code_commune_n"]!=0]

In [ ]:
df_elec.shape

Lat, Lon et adresse

Les valeurs les "moins" manquantes sont la latitude/ longitude ainsi que l"adresse, on va donc commencer par corriger tout cela

In [ ]:
# Il n'y a pas de valeurs manquantes dans consolidated latitiude
print(len(df_elec[df_elec['consolidated_latitude'].isna()]))

In [ ]:
# Il n'y a pas de valeurs manquantes dans consolidated longitude
print(len(df_elec[df_elec['consolidated_longitude'].isna()]))

In [ ]:
#M Par contre il y a des valeurs à 0 en latitude ou en longitude
print(len(df_elec.loc[(df_elec['consolidated_latitude']==0.0) | (df_elec['consolidated_longitude']==0.0),loc_col]))
df_elec.loc[(df_elec['consolidated_latitude']==0.0) | (df_elec['consolidated_longitude']==0.0),loc_col]

In [ ]:
def correct_lat_lon(dfx):
    if (dfx['consolidated_latitude']==0.0) | (dfx['consolidated_longitude']==0.0):
        dfx['lat_n'], dfx['lon_n'] = get_location_info(dfx['Adresse station'])[0:2]
    else:
        dfx['lat_n'], dfx['lon_n'] = dfx['consolidated_latitude'], dfx['consolidated_longitude']
    return dfx

In [ ]:
df_elec=df_elec.apply(lambda x: correct_lat_lon(x),axis=1)

In [ ]:
#M Par contre il y a des valeurs à 0 en latitude ou en longitude
loc_col.append('lat_n')
loc_col.append('lon_n')
print(len(df_elec.loc[(df_elec['lat_n']==0.0) | (df_elec['lon_n']==0.0),loc_col]))
df_elec.loc[(df_elec['lat_n']==0.0) | (df_elec['lon_n']==0.0),loc_col]

In [ ]:
get_location_info('2 RUE LOUIS BLERIOT 87220 FEYTIAT')

In [ ]:
for i,x in df_elec.loc[df_elec["Nom Officiel Région"].isna(),:].iterrows():
    print(i, x['Adresse station'], x['consolidated_longitude'], x['consolidated_latitude'])
    if (x['consolidated_longitude']!=0.0) & (x['consolidated_latitude']!=0.0):
        print(get_rev_location_info(x['consolidated_latitude']	,x['consolidated_longitude']))
    else:
        print(get_location_info(df_elec_2.loc[i,'Adresse station']))
    print('--------------------------------------')

In [ ]:
for i,x in df_elec.loc[(df_elec['consolidated_latitude']==0.0) | (df_elec['consolidated_longitude']==0.0),:].iterrows():
    address = x['Adresse station']
    print(address,get_location_info(address) )

In [ ]:
selected_col = [ 
        'consolidated_longitude', 'consolidated_latitude',

        'Adresse station',
                
        "Code de la commune ou de l'arrondissement","Nom de la commune ou de l'arrondissement municipal",

        'consolidated_code_postal', 'consolidated_commune','consolidated_is_lon_lat_correct',
        'Code INSEE commune',          
               ]

Regions

In [ ]:
print(len(df_elec_2[df_elec_2["Nom Officiel Région"].isna()]))

In [ ]:
df_elec_2.loc[df_elec_2["Nom Officiel Région"].isna(),loc_col]

In [ ]:
get_location_info(df_elec_2.loc[1515,'Adresse station'])

In [ ]:
get_rev_location_info(42.720000	,9.460000)

In [ ]:
            # print(infos_adresses)
            code_postal = infos_adresses[2]['city']
            code_INSEE=info_communes[0]['code']

Adresse Station

In [ ]:
print(len(df_elec[df_elec["Adresse station"].isna()]))

In [ ]:
df_elec["Adresse station"].value_counts()

In [ ]:
df_elec.dtypes

In [ ]:
df_elec.describe()

In [ ]:
df_elec[df_elec.duplicated(keep=False)]

In [ ]:
df_elec.duplicated(subset=['ID station itinérance'],keep=False)

In [ ]:
print(df_elec['Nom Officiel Région'].unique().shape)
df_elec['Nom Officiel Région'].unique()

In [ ]:
print(df_elec['Nom aménageur'].unique().shape)
df_elec['Nom aménageur'].unique()

In [ ]:
df_elec['Nom aménageur'].unique()

In [ ]:
df_elec['SIREN aménageur'].unique()

In [ ]:
df_elec['Puissance nominale'].unique()

In [ ]:
df_elec['Nombre Points de charge'].unique()

In [ ]:
df_elec.value_counts('Nombre Points de charge')

In [ ]:
df_elec.value_counts('Code INSEE commune')

In [ ]:
df_elec.shape

In [ ]:
manquantes['Nom aménageur']

In [ ]:
df_elec[df_elec['Nombre Points de charge']==505]

In [ ]:
df_elec['Date mise en service'].unique()

In [ ]:
df_elec['Date mise en service'].unique())

## Infos générales communes 2013

https://public.opendatasoft.com/explore/dataset/correspondance-code-insee-code-postal/table/

In [34]:
df_commune_info = pd.read_csv('.\\Tables_brutes\\Correspondance Code INSEE - Code Postal\\correspondance-code-insee-code-postal.csv',sep=';')

In [35]:
df_commune_info[:3]

,Code INSEE,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,ID Geofla,Code Commune,Code Canton,Code Arrondissement,Code Département,Code Région
0,59416,59190,MORBECQUE,['NORD'],['NORD-PAS-DE-CALAIS'],['Commune simple'],23.0,4455.0,2.7,"50.67689342861573, 2.536216144331492","{""coordinates"": [[[2.501239302134784, 50.63986380540941], [2.488144545060414, 50.64523101780847], [2.48046176839733, 50.65386824151298], [2.493476733962174, 50.65951289257146], [2.527123999900303, 50.67109636793778], [2.526625335431329, 50.67455839797879], [2.505966442889152, 50.68582089647341], [2.48558143554741, 50.690135329727326], [2.4795464843815322, 50.69585335871352], [2.463512379218193, 50.69702356115876], [2.464081434911729, 50.70062443146948], [2.4681751680531923, 50.720975827976595], [2.487898553770558, 50.70947397710492], [2.494004698070775, 50.714972748352515], [2.502894329541962, 50.71074623846037], [2.535394909874781, 50.70662747901899], [2.5605974628954753, 50.692998571117805], [2.581953388678889, 50.69527075174761], [2.595144440925716, 50.67308652165033], [2.602689047416944, 50.654803955750964], [2.5850118609288772, 50.643223295039014], [2.564049395245845, 50.643680954888396], [2.550493611589284, 50.65389015219217], [2.539994226805136, 50.65507219501724], [2.534836...",1237,416,30,4,59,31
1,22102,22330,LANGOURLA,"[""COTES-D'ARMOR""]",['BRETAGNE'],['Commune simple'],170.0,2165.0,0.6,"48.284641107667674, -2.415501011324659","{""coordinates"": [[[-2.416298136623701, 48.253068960500016], [-2.428604023314334, 48.258600466600065], [-2.446274744009926, 48.27509397189123], [-2.451237161550721, 48.275784952429774], [-2.439723652845889, 48.29654488940583], [-2.44229652518586, 48.308872172932], [-2.423403780549768, 48.310119456981234], [-2.4063115181742982, 48.31665625300163], [-2.406718089146765, 48.310090354234646], [-2.394306024056842, 48.29992577241352], [-2.392843955374564, 48.29370673875235], [-2.37711620400545, 48.286334212691344], [-2.3815814346943682, 48.27133176367383], [-2.409423687771154, 48.262132307265254], [-2.416298136623701, 48.253068960500016]]], ""type"": ""Polygon""}",10183,102,9,1,22,53
2,31225,31310,GOUTEVERNISSE,['HAUTE-GARONNE'],['MIDI-PYRENEES'],['Commune simple'],264.0,485.0,0.2,"43.214026301449536, 1.173765920286677","{""coordinates"": [[[1.165580460427912, 43.2003761347345], [1.158346771805673, 43.20314109807607], [1.158110248558746, 43.209786174451736], [1.168156709391063, 43.21725410115112], [1.168661834111457, 43.22615222979966], [1.184408233141277, 43.22663412525746], [1.193405395414716, 43.220252886160225], [1.18189838310755, 43.213734486220076], [1.175029141625949, 43.20104918525354], [1.165580460427912, 43.2003761347345]]], ""type"": ""Polygon""}",20555,225,27,1,31,73


In [36]:
df_commune_info.dtypes

Code INSEE              object
Code Postal             object
Commune                 object
Département             object
Région                  object
Statut                  object
Altitude Moyenne       float64
Superficie             float64
Population             float64
geo_point_2d            object
geo_shape               object
ID Geofla                int64
Code Commune             int64
Code Canton              int64
Code Arrondissement      int64
Code Département        object
Code Région              int64
dtype: object

In [37]:
df_commune_info['Département'] = df_commune_info.apply(lambda x: x['Département'].replace("['","").replace("']","").replace('["',"").replace('"]',""),axis=1)
df_commune_info['Région'] = df_commune_info.apply(lambda x: x['Région'].replace("['","").replace("']","").replace('["',"").replace('"]',""),axis=1)
df_commune_info['Statut'] = df_commune_info.apply(lambda x: x['Région'].replace("['","").replace("']","").replace('["',"").replace('"]',""),axis=1)

In [38]:
df_commune_info.shape

(36742, 17)

In [39]:
len(df_commune_info['Code INSEE'].unique())

36742

In [40]:
df_commune_info.columns

Index(['Code INSEE', 'Code Postal', 'Commune', 'Département', 'Région',
       'Statut', 'Altitude Moyenne', 'Superficie', 'Population',
       'geo_point_2d', 'geo_shape', 'ID Geofla', 'Code Commune', 'Code Canton',
       'Code Arrondissement', 'Code Département', 'Code Région'],
      dtype='object')

In [41]:
# On crée le df_code pour avoir la correspondance nom de commune avec le code INSEE
df_code = df_commune_info[['Code INSEE','Commune']]

In [13]:
# On va créer deux colonnes lat lon en splittant geo point
def split_lat_lon(s):
   try:
       lat,lon = s.split(',')
   except:
        lat,lon =np.NaN, np.NaN
   return float(lat), float(lon)
df_commune_info['lat']=df_commune_info.apply(lambda x: split_lat_lon(x['geo_point_2d'])[0],axis=1)
df_commune_info['lon']=df_commune_info.apply(lambda x: split_lat_lon(x['geo_point_2d'])[1],axis=1)

In [14]:
df_commune_info.loc[df_commune_info['Commune']=='DEMI-QUARTIER',:]

,Code INSEE,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,ID Geofla,Code Commune,Code Canton,Code Arrondissement,Code Département,Code Région,lat,lon
15479,74099,74120,DEMI-QUARTIER,HAUTE-SAVOIE,RHONE-ALPES,RHONE-ALPES,1304.0,886.0,1.0,"45.873922670726635, 6.633106273897733","{""coordinates"": [[[6.658269744218084, 45.85845...",13987,99,22,2,74,82,45.873923,6.633106


In [15]:
df_commune_info['Densité_hab_par_km2'] = df_commune_info['Population']*1000 /  (df_commune_info['Superficie'] /100)

In [16]:
df_commune_info.loc[df_commune_info['Commune']=='DEYVILLERS',:]

,Code INSEE,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,ID Geofla,Code Commune,Code Canton,Code Arrondissement,Code Département,Code Région,lat,lon,Densité_hab_par_km2
2976,88132,88000,DEYVILLERS,VOSGES,LORRAINE,LORRAINE,365.0,867.0,1.5,"48.19653045973438, 6.523290472094283","{""coordinates"": [[[6.5592820826216425, 48.2004...",6633,132,12,1,88,41,48.19653,6.52329,173.010381


In [42]:
# Valeurs uniques par colonne
df_commune_info_analyse = []
manquantes = df_commune_info.isna().sum()
for t in df_commune_info.columns:
    row = {}
    row['Variable']=t
    row['type']= df_commune_info[t].dtypes
    row['Nb_val_unique']=len(df_commune_info[t].unique())
    row['Nb_manquant']=manquantes[t]
    df_commune_info_analyse.append(row)
df_commune_info_analyse = pd.DataFrame(df_commune_info_analyse)
df_commune_info_analyse
df_commune_info_analyse.sort_values(by='Nb_manquant',ascending=False)

,Variable,type,Nb_val_unique,Nb_manquant
0,Code INSEE,object,36742,0
9,geo_point_2d,object,36742,0
15,Code Département,object,97,0
14,Code Arrondissement,int64,10,0
13,Code Canton,int64,94,0
12,Code Commune,int64,908,0
11,ID Geofla,int64,36613,0
10,geo_shape,object,36742,0
8,Population,float64,509,0
1,Code Postal,object,6101,0


In [ ]:
#df_commune_info.to_csv(r'C:\Users\nimod\OneDrive - Université de Technologie de Troyes\Fil_rouge\Tables_traitees\Infos_communes.csv',sep=';',index=False)

## Population  communes dernier recensement

In [43]:
df_pop = pd.read_csv('.\\Tables_brutes\\Population_des_communes_françaises\\population-francaise-communes_dernier_recensement.csv',sep=';')

C:\Users\nimod\AppData\Local\Temp\ipykernel_9640\1555794962.py:1: DtypeWarning: Columns (2,12,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pop = pd.read_csv('.\\Tables_brutes\\Population_des_communes_françaises\\population-francaise-communes_dernier_recensement.csv',sep=';')


In [44]:
df_pop.shape

(36750, 22)

In [20]:
df_pop.dtypes

Code région                               float64
Nom de la région                           object
Code département                           object
Code arrondissement départemental         float64
Code canton                               float64
Code commune                              float64
Nom de la commune                          object
Population municipale                     float64
Population comptée à part                 float64
Population totale                         float64
Année recensement                           int64
Année utilisation                           int64
Code INSEE (commune ou arrondissement)     object
Superficie de la commune                  float64
Statut                                     object
Code INSEE de la commune                   object
Nom de la commune IGN                      object
Nom du département IGN                     object
Nom de la région.1                         object
Code EPCI                                  object


In [21]:
df_pop[:3]

,Code région,Nom de la région,Code département,Code arrondissement départemental,Code canton,Code commune,Nom de la commune,Population municipale,Population comptée à part,Population totale,Année recensement,Année utilisation,Code INSEE (commune ou arrondissement),Superficie de la commune,Statut,Code INSEE de la commune,Nom de la commune IGN,Nom du département IGN,Nom de la région.1,Code EPCI,Code EPCI.1,EPCI
0,84.0,Auvergne-Rhône-Alpes,01,2.0,8.0,1.0,L' Abergement-Clémenciat,767.0,18.0,785.0,2016,2019,01001,15.65,Commune,01001,NaN,NaN,NaN,200035210,200035210,CC Chalaronne Centre
1,84.0,Auvergne-Rhône-Alpes,01,1.0,1.0,2.0,L' Abergement-de-Varey,243.0,1.0,244.0,2016,2019,01002,9.13,Commune,01002,L'Abergement-de-Varey,AIN,AUVERGNE-RHONE-ALPES,240100883,240100883,CC de la Plaine de l'Ain
2,84.0,Auvergne-Rhône-Alpes,01,1.0,1.0,4.0,Ambérieu-en-Bugey,14081.0,437.0,14518.0,2016,2019,01004,24.49,Commune,01004,Ambérieu-en-Bugey,AIN,AUVERGNE-RHONE-ALPES,240100883,240100883,CC de la Plaine de l'Ain


In [22]:

Code région	Nom de la région	Code département	Code arrondissement départemental	Code canton	Code commune	Nom de la commune	Population municipale	Population comptée à part	Population totale	Année recensement	Année utilisation	Code INSEE (commune ou arrondissement)	Superficie de la commune	Statut	Code INSEE de la commune	Nom de la commune IGN	Nom du département IGN	Nom de la région.1	Code EPCI	Code EPCI.1	EPCI
0	84.0	Auvergne-Rhône-Alpes	01	2.0	8.0	1.0	L' Abergement-Clémenciat	767.0	18.0	785.0	2016	2019	01001	15.65	Commune	01001	NaN	NaN	NaN	200035210	200035210	CC Chalaronne Centre
1	84.0	Auvergne-Rhône-Alpes	01	1.0	1.0	2.0	L' Abergement-de-Varey	243.0	1.0	244.0	2016	2019	01002	9.13	Commune	01002	L'Abergement-de-Varey	AIN	AUVERGNE-RHONE-ALPES	240100883	240100883	CC de la Plaine de l'Ain
2	84.0	Auvergne-Rhône-Alpes	01	1.0	1.0	4.0	Ambérieu-en-Bugey	14081.0	437.0	14518.0	2016	2019	01004	24.49	Commune	01004	Ambérieu-en-Bugey	AIN	AUVERGNE-RHONE-ALPES	240100883	240100883	CC de la Plaine de l'Ain
df_pop[df_pop['Code INSEE (commune ou arrondissement)']!=df_pop['Code INSEE de la commune']]

,Code région,Nom de la région,Code département,Code arrondissement départemental,Code canton,Code commune,Nom de la commune,Population municipale,Population comptée à part,Population totale,Année recensement,Année utilisation,Code INSEE (commune ou arrondissement),Superficie de la commune,Statut,Code INSEE de la commune,Nom de la commune IGN,Nom du département IGN,Nom de la région.1,Code EPCI,Code EPCI.1,EPCI
4522,93.0,Provence-Alpes-Côte d'Azur,13,3.0,12.0,201.0,Marseille 1er Arrondissement,40202.0,248.0,40450.0,2016,2019,13201,1.78,Arrondissement,13055,NaN,NaN,NaN,200054807,200054807,Métropole d'Aix-Marseille-Provence
4523,93.0,Provence-Alpes-Côte d'Azur,13,3.0,13.0,202.0,Marseille 2e Arrondissement,24888.0,100.0,24988.0,2016,2019,13202,3.27,Arrondissement,13055,NaN,NaN,NaN,200054807,200054807,Métropole d'Aix-Marseille-Provence
4524,93.0,Provence-Alpes-Côte d'Azur,13,3.0,98.0,203.0,Marseille 3e Arrondissement,47773.0,239.0,48012.0,2016,2019,13203,2.55,Arrondissement,13055,NaN,NaN,NaN,200054807,200054807,Métropole d'Aix-Marseille-Provence
4525,93.0,Provence-Alpes-Côte d'Azur,13,3.0,98.0,204.0,Marseille 4e Arrondissement,48074.0,371.0,48445.0,2016,2019,13204,2.89,Arrondissement,13055,NaN,NaN,NaN,200054807,200054807,Métropole d'Aix-Marseille-Provence
4526,93.0,Provence-Alpes-Côte d'Azur,13,3.0,98.0,205.0,Marseille 5e Arrondissement,46274.0,354.0,46628.0,2016,2019,13205,2.19,Arrondissement,13055,NaN,NaN,NaN,200054807,200054807,Métropole d'Aix-Marseille-Provence
4527,93.0,Provence-Alpes-Côte d'Azur,13,3.0,98.0,206.0,Marseille 6e Arrondissement,43070.0,428.0,43498.0,2016,2019,13206,2.06,Arrondissement,13055,NaN,NaN,NaN,200054807,200054807,Métropole d'Aix-Marseille-Provence
4528,93.0,Provence-Alpes-Côte d'Azur,13,3.0,23.0,207.0,Marseille 7e Arrondissement,35173.0,376.0,35549.0,2016,2019,13207,5.85,Arrondissement,13055,NaN,NaN,NaN,200054807,200054807,Métropole d'Aix-Marseille-Provence
4529,93.0,Provence-Alpes-Côte d'Azur,13,3.0,98.0,208.0,Marseille 8e Arrondissement,80724.0,1222.0,81946.0,2016,2019,13208,20.75,Arrondissement,13055,NaN,NaN,NaN,200054807,200054807,Métropole d'Aix-Marseille-Provence
4530,93.0,Provence-Alpes-Côte d'Azur,13,3.0,98.0,209.0,Marseille 9e Arrondissement,74523.0,723.0,75246.0,2016,2019,13209,63.58,Arrondissement,13055,NaN,NaN,NaN,200054807,200054807,Métropole d'Aix-Marseille-Provence
4531,93.0,Provence-Alpes-Côte d'Azur,13,3.0,98.0,210.0,Marseille 10e Arrondissement,56191.0,423.0,56614.0,2016,2019,13210,10.75,Arrondissement,13055,NaN,NaN,NaN,200054807,200054807,Métropole d'Aix-Marseille-Provence


In [23]:
df_pop[df_pop['Code EPCI']!=df_pop['Code EPCI.1']].shape

(26, 22)

In [24]:
df_pop[df_pop['Code INSEE (commune ou arrondissement)']=='13055']

,Code région,Nom de la région,Code département,Code arrondissement départemental,Code canton,Code commune,Nom de la commune,Population municipale,Population comptée à part,Population totale,Année recensement,Année utilisation,Code INSEE (commune ou arrondissement),Superficie de la commune,Statut,Code INSEE de la commune,Nom de la commune IGN,Nom du département IGN,Nom de la région.1,Code EPCI,Code EPCI.1,EPCI


In [45]:
# Valeurs uniques par colonne
df_pop_analyse = []
manquantes = df_pop.isna().sum()
for t in df_pop.columns:
    row = {}
    row['Variable']=t
    row['type']= df_pop[t].dtypes
    row['Nb_val_unique']=len(df_pop[t].unique())
    row['Nb_manquant']=manquantes[t]
    df_pop_analyse.append(row)
df_pop_analyse = pd.DataFrame(df_pop_analyse)
df_pop_analyse
# df_pop_analyse.sort_values(by='Nb_manquant',ascending=False)

,Variable,type,Nb_val_unique,Nb_manquant
0,Code région,float64,34,0
1,Nom de la région,object,36,0
2,Code département,object,102,0
3,Code arrondissement départemental,float64,10,17
4,Code canton,float64,59,37
5,Code commune,float64,908,0
6,Nom de la commune,object,34250,0
7,Population municipale,float64,5845,0
8,Population comptée à part,float64,793,0
9,Population totale,float64,5942,0


In [26]:
# On filtre pour ne garder que les champs que l'on veut
# On va garder que les codes des communes et des arrondissements
df_pop = df_pop[['Code région', 'Nom de la région', 'Code département',
         'Code commune',
       'Nom de la commune', 'Population municipale',
       'Population comptée à part', 'Population totale', 'Année recensement',
       'Année utilisation', 'Code INSEE (commune ou arrondissement)',
       'Superficie de la commune', 'Statut', 
       'Code EPCI', 'EPCI']]

In [27]:
df_pop['Densité_hab_par_km2'] = df_pop['Population municipale'] /  df_pop['Superficie de la commune']

In [28]:
df_pop.loc[df_pop['Nom de la commune']=='Deyvillers',:]

,Code région,Nom de la région,Code département,Code commune,Nom de la commune,Population municipale,Population comptée à part,Population totale,Année recensement,Année utilisation,Code INSEE (commune ou arrondissement),Superficie de la commune,Statut,Code EPCI,EPCI,Densité_hab_par_km2
35173,44.0,Grand Est,88,132.0,Deyvillers,1446.0,49.0,1495.0,2016,2019,88132,8.67,Commune,200034361,CA d'Épinal,166.782007


In [29]:
df_pop.columns

Index(['Code région', 'Nom de la région', 'Code département', 'Code commune',
       'Nom de la commune', 'Population municipale',
       'Population comptée à part', 'Population totale', 'Année recensement',
       'Année utilisation', 'Code INSEE (commune ou arrondissement)',
       'Superficie de la commune', 'Statut', 'Code EPCI', 'EPCI',
       'Densité_hab_par_km2'],
      dtype='object')

In [ ]:
# df_pop.to_csv(r'C:\Users\nimod\OneDrive - Université de Technologie de Troyes\Fil_rouge\Tables_traitees\Population-Superficie_communes.csv',sep=';',index=False)

## Communes adjacentes

In [46]:
df_commune_adj = pd.read_csv('.\\Tables_brutes\\Communes_adjacentes\\communes_adjacentes_2022.csv',sep=',')

In [47]:
df_commune_adj.shape

(34990, 7)

In [32]:
df_commune_adj[:3]

,insee,nom,nb_voisins,insee_voisins,noms_voisins,cap_voisins,type
0,01001,L'Abergement-Clémenciat,6,01412|01093|01028|01146|01351|01188,Sulignat|Châtillon-sur-Chalaronne|Baneins|Domp...,48|144|203|234|260|322,NaN
1,01002,L'Abergement-de-Varey,6,01056|01277|01384|01007|01363|01199,Boyeux-Saint-Jérôme|Nivollet-Montgriffon|Saint...,43|115|172|265|302|344,NaN
2,01004,Ambérieu-en-Bugey,7,01384|01421|01041|01345|01089|01007|01149,Saint-Rambert-en-Bugey|Torcieu|Bettant|Saint-D...,102|151|190|250|284|347|360,NaN


## Prix_vente_m2_communes

In [55]:
df_commune_immo = pd.read_csv('.\\Tables_brutes\\Prix_moyen_m2_vente\\dvf-communes-2019.csv',sep=';')

In [56]:
df_commune_immo.dtypes

ID               object
INSEE_COM        object
INSEE_DEP        object
INSEE_REG         int64
CODE_EPCI       float64
NOM_COM_M        object
POPULATION        int64
Nb_Ventes       float64
PrixMoyen_M2    float64
dtype: object

In [57]:
df_commune_immo.shape

(34886, 9)

In [58]:
df_commune_immo[:5]

,ID,INSEE_COM,INSEE_DEP,INSEE_REG,CODE_EPCI,NOM_COM_M,POPULATION,Nb_Ventes,PrixMoyen_M2
0,COMMUNE_0000000009760754,32216,32,76,200072320.0,LOURTIES MONBRUN,152,NaN,NaN
1,COMMUNE_0000000009756744,47033,47,75,200036523.0,BOUDY DE BEAUREGARD,412,4.0,1421.0
2,COMMUNE_0000000009760252,32009,32,76,243200425.0,ARMOUS ET CAU,86,NaN,NaN
3,COMMUNE_0000000009754403,38225,38,84,243801024.0,AUTRANS MEAUDRE EN VERCORS,2969,47.0,2667.0
4,COMMUNE_0000000009727903,62890,62,32,200044030.0,WILLEMAN,182,3.0,967.0


In [59]:
df_commune_immo.columns

Index(['ID', 'INSEE_COM', 'INSEE_DEP', 'INSEE_REG', 'CODE_EPCI', 'NOM_COM_M',
       'POPULATION', 'Nb_Ventes', 'PrixMoyen_M2'],
      dtype='object')

In [60]:
df_commune_immo.dtypes

ID               object
INSEE_COM        object
INSEE_DEP        object
INSEE_REG         int64
CODE_EPCI       float64
NOM_COM_M        object
POPULATION        int64
Nb_Ventes       float64
PrixMoyen_M2    float64
dtype: object

In [61]:
df_commune_immo[df_commune_immo.INSEE_COM=='88132']

,ID,INSEE_COM,INSEE_DEP,INSEE_REG,CODE_EPCI,NOM_COM_M,POPULATION,Nb_Ventes,PrixMoyen_M2
30853,COMMUNE_0000000009740362,88132,88,44,200068757.0,DEYVILLERS,1446,10.0,1728.0


In [62]:
df_commune_immo['nb_digits_code_INSEE'] = df_commune_immo['INSEE_COM'].str.len()
df_commune_immo['nb_digits_code_INSEE'].unique()

array([5, 4], dtype=int64)

In [63]:
# On va corriger le code commune dans ce fichier, car le premier 0 a été oublié
def add_0(x):
    if len(str(x))==4:
        x = "0"+str(x)
    else:
        x =str(x)
    return x 
df_commune_immo['code_commune_n'] = df_commune_immo.apply(lambda x: add_0(x['INSEE_COM']), axis=1)

In [64]:
# Valeurs uniques par colonne
df_commune_immo_analyse = []
manquantes = df_commune_immo.isna().sum()
for t in df_commune_immo.columns:
    row = {}
    row['Variable']=t
    row['type']= df_commune_immo[t].dtypes
    row['Nb_val_unique']=len(df_commune_immo[t].unique())
    row['Nb_manquant']=manquantes[t]
    df_commune_immo_analyse.append(row)
df_commune_immo_analyse = pd.DataFrame(df_commune_immo_analyse)
df_commune_immo_analyse.sort_values(by='Nb_manquant',ascending=False)

,Variable,type,Nb_val_unique,Nb_manquant
7,Nb_Ventes,float64,554,8064
8,PrixMoyen_M2,float64,3709,8064
4,CODE_EPCI,float64,1237,49
0,ID,object,34886,0
1,INSEE_COM,object,34886,0
2,INSEE_DEP,object,96,0
3,INSEE_REG,int64,13,0
5,NOM_COM_M,object,32432,0
6,POPULATION,int64,5736,0
9,nb_digits_code_INSEE,int64,2,0


In [43]:
df_commune_immo['nb_digits_code_INSEE'] = df_commune_immo['code_commune_n'].str.len()
df_commune_immo['nb_digits_code_INSEE'].unique()

array([5], dtype=int64)

In [44]:
#df_commune_immo.to_csv(r'C:\Users\nimod\OneDrive - Université de Technologie de Troyes\Fil_rouge\Tables_traitees\PrixMoy_m2_avec_code_commune.csv', sep=';',index=False)

## Stations carburants en France à J-7

https://public.opendatasoft.com/explore/dataset/prix_des_carburants_j_7/information/?sort=update

In [65]:
df_carb = pd.read_csv('.\\Tables_brutes\\Prix des carburants - J-7\\prix_des_carburants_j_7.csv',sep=';')

In [66]:
df_carb[:10]

,Identifiant,Code Postal,Présence,Adresse,Ville,Automate 24-24,Timetable,Carburant,Rupture,Mise à jour,Prix Gazole,Prix SP95,Prix SP98,Prix GPLc,Prix E10,Prix E85,Services,Marque,Nom,Geo Point
0,14730002,14730,A,Autoroute de Normandie,GIBERVILLE,NaN,NaN,Gazole,"SP95,E85,GPLc",2020-01-06T12:44:00+01:00,1.624000,NaN,1.754000,NaN,1.644000,NaN,NaN,Esso,ESSO GIBERVILLE,"49.16711, -0.28803"
1,84550001,84550,A,A7 AIRE DE MORNAS VILLAGE,MORNAS,NaN,NaN,"Gazole,SP95,E85,GPLc,E10,SP98",NaN,2020-01-12T09:51:06+01:00,1.544000,1.624,1.654000,0.897,1.594000,0.749000,"Restauration à emporter,Carburant additivé,Toilettes publiques,Douches,Boutique alimentaire,Station de gonflage,Espace bébé,Boutique non alimentaire,Piste poids lourds,DAB (Distributeur automatique de billets),Lavage automatique",Carrefour,AIRE DE MORNAS VILLAGE (LES CROUSILLES),"44.2036953693, 4.72398720666"
2,67202004,67202,R,"RN 4 Route de Saverne, Station Total Access",WOLFISHEIM,Oui,"{""Dimanche"": {""fermeture"": ""20.00"", ""ouvert"": 1, ""ouverture"": ""08.00""}, ""Jeudi"": {""fermeture"": ""20.00"", ""ouvert"": 1, ""ouverture"": ""07.00""}, ""Lundi"": {""fermeture"": ""20.00"", ""ouvert"": 1, ""ouverture"": ""07.00""}, ""Mardi"": {""fermeture"": ""20.00"", ""ouvert"": 1, ""ouverture"": ""07.00""}, ""Mercredi"": {""fermeture"": ""20.00"", ""ouvert"": 1, ""ouverture"": ""07.00""}, ""Samedi"": {""fermeture"": ""20.00"", ""ouvert"": 1, ""ouverture"": ""08.00""}, ""Vendredi"": {""fermeture"": ""20.00"", ""ouvert"": 1, ""ouverture"": ""07.00""}}","E85,E10,SP98","GPLc,SP95",2022-11-22T08:52:37+01:00,0.001884,NaN,0.001765,NaN,0.001675,0.000929,"Relais colis,Boutique alimentaire,Boutique non alimentaire",NaN,NaN,NaN
3,37600006,37600,R,"105, Rue SAINT-JACQUES",LOCHES,Oui,"{""Mardi"": {""ouverture"": ""07.30"", ""fermeture"": ""19.00"", ""ouvert"": 1}, ""Samedi"": {""ouverture"": ""08.00"", ""fermeture"": ""12.00"", ""ouvert"": 1}, ""Vendredi"": {""ouverture"": ""07.30"", ""fermeture"": ""19.00"", ""ouvert"": 1}, ""Jeudi"": {""ouverture"": ""07.30"", ""fermeture"": ""19.00"", ""ouvert"": 1}, ""Lundi"": {""ouverture"": ""07.30"", ""fermeture"": ""19.00"", ""ouvert"": 1}, ""Dimanche"": {""ouvert"": 0}, ""Mercredi"": {""ouverture"": ""07.30"", ""fermeture"": ""19.00"", ""ouvert"": 1}}",SP95,"SP98,GPLc,E85,E10",2019-04-24T08:06:30+02:00,1.510000,1.620,NaN,NaN,NaN,NaN,"Toilettes publiques,Station de gonflage,Vente de gaz domestique (Butane, Propane),Vente de fioul domestique",Indépendant sans enseigne,GLEMAREC SARL,"47.13964, 1.00018"
4,38650002,38650,R,1 route de sinard,MONESTIER-DE-CLERMONT,NaN,NaN,SP98,"SP95,E85,GPLc",2019-02-22T18:31:10+01:00,1.530000,NaN,1.630000,NaN,1.520000,NaN,"Carburant additivé,Toilettes publiques,Station de gonflage,Boutique non alimentaire,Vente de gaz domestique (Butane, Propane),Piste poids lourds,DAB (Distributeur automatique de billets)",Total,Snga,"44.92634, 5.6382200000000005"
5,25430002,25430,R,7 rue de Besançon,SANCEY-LE-GRAND,NaN,NaN,"Gazole,SP95",NaN,2019-04-12T12:49:43+02:00,1.520000,1.620,NaN,NaN,NaN,NaN,Carburant additivé,Avia,GRUT Jean-Louis,"47.2971, 6.5808"
6,29160001,29160,R,9 route de Chateaulin TAL AR GROAS,CROZON,Non,"{""Mardi"": {""ouverture"": ""08.00"", ""fermeture"": ""19.00"", ""ouvert"": 1}, ""Samedi"": {""ouverture"": ""01.00"", ""fermeture"": ""01.00"", ""ouvert"": 0}, ""Vendredi"": {""ouverture"": ""08.00"", ""fermeture"": ""19.00"", ""ouvert"": 1}, ""Jeudi"": {""ouverture"": ""08.00"", ""fermeture"": ""19.00"", ""ouvert"": 1}, ""Lundi"": {""ouverture"": ""08.00"", ""fermeture"": ""19.00"", ""ouvert"": 1}, ""Dimanche"": {""ouverture"": ""01.00"", ""fermeture"": ""01.00"", ""ouvert"": 0}, ""Mercredi"": {""ouverture"": ""08.00"", ""fermeture"": ""19.00"", ""ouvert"": 1}}",SP98,"E85,GPLc,SP95",2019-02-25T18:14:21+01:00,1.570000,NaN,1.630000,NaN,1.550000,NaN,"Carburant additivé,Station de gonflage,Boutique non alimentaire,Vente de gaz domestique (Butane, Propane),Piste poids lourds,DAB (Distributeur automatique de billets),Vente de 

In [67]:
df_carb.columns

Index(['Identifiant', 'Code Postal', 'Présence', 'Adresse', 'Ville',
       'Automate 24-24', 'Timetable', 'Carburant', 'Rupture', 'Mise à jour',
       'Prix Gazole', 'Prix SP95', 'Prix SP98', 'Prix GPLc', 'Prix E10',
       'Prix E85', 'Services', 'Marque', 'Nom', 'Geo Point'],
      dtype='object')

In [68]:
df_carb.shape

(10962, 20)

In [69]:
df_carb['Identifiant'].unique().shape

(10962,)

In [70]:
df_carb['Identifiant'].unique()

array([14730002, 84550001, 67202004, ...,  3150007,  3160001,  3290001],
      dtype=int64)

In [71]:
# On va créer deux colonnes lat lon en splittant geo point
def split_lat_lon(s):
   try:
       lat,lon = s.split(',')
   except:
        lat,lon =np.NaN, np.NaN
   return float(lat), float(lon)
df_carb['lat']=df_carb.apply(lambda x: split_lat_lon(x['Geo Point'])[0],axis=1)
df_carb['lon']=df_carb.apply(lambda x: split_lat_lon(x['Geo Point'])[1],axis=1)

In [72]:
# Valeurs uniques par colonne
df_carb_analyse = []
manquantes = df_carb.isna().sum()
for t in df_carb.columns:
    row = {}
    row['Variable']=t
    row['type']= df_carb[t].dtypes
    row['Nb_val_unique']=len(df_carb[t].unique())
    row['Nb_manquant']=manquantes[t]
    df_carb_analyse.append(row)
df_carb_analyse = pd.DataFrame(df_carb_analyse)
df_carb_analyse.sort_values(by='Nb_manquant',ascending=False)

,Variable,type,Nb_val_unique,Nb_manquant
13,Prix GPLc,float64,279,9427
15,Prix E85,float64,352,7342
11,Prix SP95,float64,593,6769
20,lat,float64,7196,3703
19,Geo Point,object,7228,3703
21,lon,float64,7198,3703
17,Marque,object,74,3688
18,Nom,object,5652,3688
14,Prix E10,float64,709,3028
6,Timetable,object,1609,2756


In [ ]:
## On commence par rechercher dans le df_code s'il y a des villes déjà enregistrées
# df_carb = pd.merge(df_carb, df_code, left_on='Ville', right_on='Commune', how='left')
# df_carb['code_commune_n']=df_carb['Code INSEE']

In [ ]:
# Ensuite on Recherche par ville sur l'API gouv.fr 
def code(x):
    if pd.isna(x['code_commune_n']):
        c = info_communes(x['Ville'])[0]['code']
    else:
        c =  x['code_commune_n']
    return c
df_carb['code_commune_n'] = df_carb.apply(lambda x: code(x),axis=1)

In [ ]:
# ATTENTION CODE LONG
# Ensuite on recherche par lat lon 
for i, x in df_carb[(df_carb['code_commune_n'].isna()) & (df_carb['lat'].notna()) & (df_carb['lon'].notna())].iterrows():
    print(i)
    try: 
        df_carb.loc[i,'code_commune_n'] = get_rev_commune(df_carb.loc[i,'lat'],df_carb.loc[i,'lon'])['features'][0]['properties']['citycode']
    except:
        pass

In [ ]:
# ATTENTION CODE LONG
#Pour ceux qui manque toujours on va essayer la recherche avec  l'adresse, on pourrait aussi essayer l'API google map mais c'est payant
for i, x in df_carb[(df_carb['code_commune_n'].isna())].iterrows():
    print(i,f"{x['Adresse']} {x['Code Postal']} {x['Ville']}")
    df_carb.loc[i,'lat'],df_carb.loc[i,'lon'],_,_ = get_location_info(f"{x['Adresse']} {x['Code Postal']} {x['Ville']}")
    time.sleep(1)

In [73]:
get_location_info('56 AVE DU DR PICAUD 6150 CANNES LA BOCCA')

(None, None, None, None)

In [74]:
get_location_info('33 rue du commandant René Mouchotte 75014 PARIS')

(48.8375511,
 2.3187555,
 {'road': 'Rue du Commandant René Mouchotte',
  'city_block': 'Quartier de Plaisance',
  'neighbourhood': 'Paris 14e Arrondissement',
  'suburb': 'Paris',
  'city_district': 'Paris',
  'city': 'Paris',
  'ISO3166-2-lvl6': 'FR-75',
  'state': 'Île-de-France',
  'ISO3166-2-lvl4': 'FR-IDF',
  'region': 'France métropolitaine',
  'postcode': '75014',
  'country': 'France',
  'country_code': 'fr'},
 'France')

In [77]:
get_rev_commune(48.8375511,2.3187555)

{'type': 'FeatureCollection',
 'version': 'draft',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [2.318537, 48.83775]},
   'properties': {'label': '40 Rue du Commandant René Mouchotte 75014 Paris',
    'score': 0.9999997024231708,
    'housenumber': '40',
    'id': '75114_2236_00040',
    'name': '40 Rue du Commandant René Mouchotte',
    'postcode': '75014',
    'citycode': '75114',
    'x': 649981.04,
    'y': 6859960.12,
    'city': 'Paris',
    'district': 'Paris 14e Arrondissement',
    'context': '75, Paris, Île-de-France',
    'type': 'housenumber',
    'importance': 0.71722,
    'street': 'Rue du Commandant René Mouchotte',
    'distance': 27}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [2.31865, 48.837176]},
   'properties': {'label': 'Place de Catalogne 75014 Paris',
    'score': 0.9999992811923678,
    'id': '75114_1604',
    'name': 'Place de Catalogne',
    'postcode': '75014',
    'citycode': '75114',
    'x'

In [76]:
info_communes('Paris')

81202


[{'nom': 'Parisot',
  'code': '81202',
  'codeDepartement': '81',
  'siren': '218102028',
  'codeEpci': '200066124',
  'codeRegion': '76',
  'codesPostaux': ['81310'],
  'population': 963,
  '_score': 0.5093474202233138},
 {'nom': 'Parisot',
  'code': '82137',
  'codeDepartement': '82',
  'siren': '218201374',
  'codeEpci': '248200107',
  'codeRegion': '76',
  'codesPostaux': ['82160'],
  'population': 555,
  '_score': 0.5093474202233138},
 {'nom': 'Paris',
  'code': '75056',
  'codeDepartement': '75',
  'siren': '217500016',
  'codeEpci': '200054781',
  'codeRegion': '11',
  'codesPostaux': ['75001',
   '75002',
   '75003',
   '75004',
   '75005',
   '75006',
   '75007',
   '75008',
   '75009',
   '75010',
   '75011',
   '75012',
   '75013',
   '75014',
   '75015',
   '75016',
   '75017',
   '75018',
   '75019',
   '75020',
   '75116'],
  'population': 2145906,
  '_score': 0.4327063840922378},
 {'nom': 'Fontenay-en-Parisis',
  'code': '95241',
  'codeDepartement': '95',
  'siren': '21

In [ ]:
df_carb[(df_carb['code_commune_n'].isna())]

In [ ]:
#df_carb.to_csv(r'C:\Users\nimod\OneDrive - Université de Technologie de Troyes\Fil_rouge\Tables_traitees\Stations_carburant_avec_code_commune.csv', sep=';',index=False)

In [ ]:

#df_carb['code_commune_n']= df_carb.apply(lambda x: info_communes(x['Ville'])[0]['code'],axis=1)

In [ ]:
info_communes(df_carb.loc[5,'Ville'])

In [ ]:
get_rev_location_info(df_carb.loc[0,'lat'],df_carb.loc[0,'lon'])

In [ ]:
get_rev_commune(df_carb.loc[7702,'lat'],df_carb.loc[7702,'lon'])['features'][0]['properties']['citycode']

In [ ]:
get_rev_commune(df_carb.loc[7702,'lat'],df_carb.loc[7702,'lon'])

## Europe - Railway Station

https://public.opendatasoft.com/explore/dataset/europe-railway-station/information/

In [ ]:
df_rail = pd.read_csv(glob.glob('.\\Europe Railway Station\\*.csv')[0],sep=';')

In [ ]:
df_rail[:5]

In [ ]:
df_rail['ICC'].unique()

In [ ]:
df_rail.columns

In [ ]:
df_rail.shape

## Base Nationale des Lieux de Covoiturage en France

https://public.opendatasoft.com/explore/dataset/base-nationale-des-lieux-de-covoiturage-france/information/

In [ ]:
df_cov = pd.read_csv(glob.glob('.\\Base Nationale des Lieux de Covoiturage en France\\*.csv')[0],sep=';')

In [ ]:
df_cov[:5]

In [ ]:
df_cov.columns

In [ ]:
df_cov.shape

## Base nationale des lieux de stationnement en France

https://public.opendatasoft.com/explore/dataset/mobilityref-france-base-nationale-des-lieux-de-stationnement/table/

In [ ]:
df_stat = pd.read_csv(glob.glob('.\\Base nationale des lieux de stationnement en France\\*.csv')[0],sep=';')

In [ ]:
df_stat[:5]

In [ ]:
df_stat.columns

In [ ]:
df_stat.shape